In [0]:
def apply_discount(amount: float, rate: float = 0.1):
    """
    Applies discount to the amount.
    """
    return amount * (1 - rate)

dbutils.widgets.text("amount", "100")
amount = float(dbutils.widgets.get("amount"))

discounted = apply_discount(amount)
print(f"Discounted amount: {discounted}")


In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()
%restart_python

In [0]:
%python
   import mlflow
   import json
   from databricks.sdk import WorkspaceClient
   from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient

   # Get an OpenAI client configured to connect to Databricks model serving endpoints
   openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

   # Enable automatic tracing for easier debugging
   mlflow.openai.autolog()

   # Load Databricks built-in tools (Python code interpreter)
   client = DatabricksFunctionClient()
   builtin_tools = UCFunctionToolkit(function_names=["system.ai.python_exec"], client=client).tools
   for tool in builtin_tools:
       del tool["function"]["strict"]

   def call_tool(tool_name, parameters):
       if tool_name == "system__ai__python_exec":
           return DatabricksFunctionClient().execute_function("system.ai.python_exec", parameters=parameters)
       raise ValueError(f"Unknown tool: {tool_name}")

   def run_agent(prompt):
       """
       Send a user prompt to the LLM and return a list of LLM response messages.
       The LLM is allowed to call the code interpreter tool, if needed, to respond to the user.
       """
       result_msgs = []
       response = openai_client.chat.completions.create(
           model="databricks-claude-3-7-sonnet",
           messages=[{"role": "user", "content": prompt}],
           tools=builtin_tools,
       )
       msg = response.choices[0].message
       result_msgs.append(msg.to_dict())
       # If the model executed a tool, call it
       if msg.tool_calls:
           call = msg.tool_calls[0]
           tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
           result_msgs.append({"role": "tool", "content": tool_result.value, "name": call.function.name, "tool_call_id": call.id})
       return result_msgs